In [5]:
import boto3
from pprint import pprint
client = boto3.client('batch')

# Register job definition

In [4]:
client = boto3.client('batch')
resp_reg_job = client.register_job_definition(
    jobDefinitionName='naturalconversion',
    type='container',
    containerProperties={
        'image': '273676533378.dkr.ecr.us-east-1.amazonaws.com/sbtn/naturalconversion:latest',
        'command': [
            'write_csvs_for_postgres',
        ],
        'resourceRequirements': [
            {
                'value': '256000',
                'type': 'MEMORY'
            },
            {
                'value': '32',
                'type': 'VCPU'
            },
        ]
    },
    timeout={
        'attemptDurationSeconds': 3600*12
    },
    platformCapabilities=[
        'EC2',
    ]
)
pprint(resp_reg_job)

{'ResponseMetadata': {'HTTPHeaders': {'access-control-allow-origin': '*',
                                      'access-control-expose-headers': 'X-amzn-errortype,X-amzn-requestid,X-amzn-errormessage,X-amzn-trace-id,X-amz-apigw-id,date',
                                      'connection': 'keep-alive',
                                      'content-length': '147',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 28 Feb 2022 03:15:32 GMT',
                                      'x-amz-apigw-id': 'OO6tME4-IAMF7kw=',
                                      'x-amzn-requestid': '0de5c0de-a5b9-4103-9067-8dd04fb73a23',
                                      'x-amzn-trace-id': 'Root=1-621c3e54-22f668316dbe77055099c6c2'},
                      'HTTPStatusCode': 200,
                      'RequestId': '0de5c0de-a5b9-4103-9067-8dd04fb73a23',
                      'RetryAttempts': 0},
 'jobDefinitionArn': 'arn:aws:batch:us-ea

# Run Job

In [10]:
job_name = "cropland_match"
n_cpus = "2"
memory = "16000"
array_size = 1296
timeout = 3600*6
job_queue ='spot_standard_disk'

#job_name = "natural_conversion"
#n_cpus = "62"
#memory = "350000"
#array_size = 1
#timeout = 3600*12
#job_queue ='spot_standard_disk'

#job_queue = 'ondemand_large_disk'
#job_queue ='spot_standard_disk'
#job_queue ='spot_fleet_1TB-io2-disk'
#job_queue ='ondemand_fleet_1TB-io2-disk'

job_params = dict(
    jobName=f'{job_name}',
    jobDefinition='naturalconversion',
    jobQueue=job_queue,
    containerOverrides={
        'resourceRequirements': [
            {
                'value': n_cpus,
                'type': 'VCPU'
            },
            {
                'value': memory,
                'type': 'MEMORY'
            }
        ],
        'command': [
            f'{job_name}'
        ],
    },
    timeout={
        'attemptDurationSeconds': timeout
    },
    retryStrategy={ 
        "attempts": 4,
        "evaluateOnExit": 
        [
            {
                "onStatusReason" :"Host EC2*",  # includes spot instance reclaim
                "action": "RETRY"
            },
            {
                "onStatusReason" :"CannotInspectContainer*",  # docker errors usually due to i/o
                "action": "RETRY"
            },
            {
                "onStatusReason" :"OutOfMemoryError*",
                "action": "RETRY"
            },
            {
                "onReason" : "*",
                "action": "EXIT"
            }
        ]
    }
)
if array_size > 1:
    job_params.update(dict(
        arrayProperties={
            'size': array_size
        }
    ))
resp_submit = client.submit_job(**job_params)
pprint(resp_submit)

{'ResponseMetadata': {'HTTPHeaders': {'access-control-allow-origin': '*',
                                      'access-control-expose-headers': 'X-amzn-errortype,X-amzn-requestid,X-amzn-errormessage,X-amzn-trace-id,X-amz-apigw-id,date',
                                      'connection': 'keep-alive',
                                      'content-length': '164',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 04 Mar 2022 03:19:56 GMT',
                                      'x-amz-apigw-id': 'OcHGfFCyoAMFn9Q=',
                                      'x-amzn-requestid': 'a67cf1db-c4ab-495d-8cdd-78e5e0f7dce5',
                                      'x-amzn-trace-id': 'Root=1-6221855c-6b46f17c6897d5d442a4cfc9'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'a67cf1db-c4ab-495d-8cdd-78e5e0f7dce5',
                      'RetryAttempts': 0},
 'jobArn': 'arn:aws:batch:us-east-1:27367